# Your info

Full name: M.Amin.Ghasvari

Student ID: 97521432

Notice: **You can add new cells for each part.**

# Q1

In [11]:
import math
import random
from random import randrange


class SalesmanSolver:

    def __init__(self, genes, length, decode, fitness):
        self.genes = genes
        self.individuals_length = length
        self.decode = decode
        self.fitness = fitness

    @staticmethod
    def inversion_mutation(chromosome_aux):
        chromosome = chromosome_aux
        index1 = randrange(0, len(chromosome))
        index2 = randrange(index1, len(chromosome))
        chromosome_mid = chromosome[index1:index2]
        chromosome_mid.reverse()
        chromosome_result = chromosome[0:index1] + chromosome_mid + chromosome[index2:]

        return chromosome_result

    def mutation(self, chromosome, prob):
        aux = []
        for _ in range(len(chromosome)):
            if random.random() < prob:
                aux = self.inversion_mutation(chromosome)
        return aux

    def crossover(self, parent1, parent2):
        def process_gen_repeated(copy_child1, copy_child2):
            count1 = 0
            for gen1 in copy_child1[:pos]:
                repeat = copy_child1.count(gen1)
                if repeat > 1:
                    count2 = 0
                    for gen2 in parent1[pos:]:
                        if gen2 not in copy_child1:
                            child1[count1] = parent1[pos:][count2]
                        count2 += 1
                count1 += 1

            count1 = 0
            for gen1 in copy_child2[:pos]:
                repeat = copy_child2.count(gen1)
                if repeat > 1:
                    count2 = 0
                    for gen2 in parent2[pos:]:
                        if gen2 not in copy_child2:
                            child2[count1] = parent2[pos:][count2]
                        count2 += 1
                count1 += 1

            return [child1, child2]

        pos = random.randrange(1, self.individuals_length - 1)
        child1 = parent1[:pos] + parent2[pos:]
        child2 = parent2[:pos] + parent1[pos:]

        return process_gen_repeated(child1, child2)


def decode_node(chromosome):
    l = []
    for i in chromosome:
        l.append(houses.get(i))
    return l


def penalty(chromosome):
    actual = chromosome
    value_penalty = 0
    for i in actual:
        times = actual.count(i)
        if times > 1:
            value_penalty += 100 * abs(times - len(actual))
    return value_penalty


def fit_salesman(chromosome):
    actualChromosome = list(chromosome)
    fitness_value = 0
    count = 0

    penalty_value = penalty(actualChromosome)

    for i in chromosome:
        if count == 6:
            next_house = actualChromosome[0]
        else:
            temp = count + 1
            next_house = actualChromosome[temp]

        w = distances.get(i)
        fitness_value += w[next_house] + 50 * penalty_value
        count += 1

    return fitness_value


def initial_population(problem, size):
    def generate_chromosome():
        chromosome = []
        for i in problem.genes:
            chromosome.append(i)
        random.shuffle(chromosome)
        return chromosome

    return [generate_chromosome() for _ in range(size)]


def cross_parents(problem, parents):
    children = []
    for i in range(0, len(parents), 2):
        children.extend(problem.crossover(parents[i], parents[i + 1]))
    return children


def tournament_selection(problem, population, n, k, opt):
    winners = []

    for _ in range(int(n)):
        elements = random.sample(population, k)
        winners.append(opt(elements, key=problem.fitness))
    return winners


def mutate(problem, population, prob):
    for i in population:
        problem.mutation(i, prob)
    return population


def new_generation_t(problem, k, opt, population, n_parents, n_directs, prob_mutate):
    directs = tournament_selection(problem, population, n_directs, k, opt)
    crosses = cross_parents(problem,
                            tournament_selection(problem, population, n_parents, k, opt))
    mutations = mutate(problem, crosses, prob_mutate)
    new_generation = directs + mutations

    return new_generation


def genetic_algorithm_t(problem, k, opt, num_gen, size, ratio_cross, prob_mutate):
    population = initial_population(problem, size)
    n_parents = round(size * ratio_cross)
    n_parents = (n_parents if n_parents % 2 == 0 else n_parents - 1)
    n_directs = int(size - n_parents)

    for _ in range(num_gen):
        population = new_generation_t(problem, k, opt, population, n_parents, n_directs, prob_mutate)

    best_chromosome = opt(population, key=problem.fitness)
    genotype = problem.decode(best_chromosome)
    print("Chromosome: {}\nSolution: {}".format(best_chromosome, (genotype, problem.fitness(best_chromosome))))
    return genotype, problem.fitness(best_chromosome)


In [12]:
INF = math.inf
POPULATION_SIZE = 10
CROSS_PROB = 0.02
RATIO = 0.5

if __name__ == "__main__":
    num_of_houses = 7
    houses = {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7'}
    distances = {
        0: [INF, 12, 10, INF, INF, INF, 12],
        1: [12, INF, 8, 12, INF, INF, INF],
        2: [10, 8, INF, 11, 3, INF, 9],
        3: [INF, 12, 11, INF, 11, 10, INF],
        4: [INF, INF, 3, 11, INF, 6, 7],
        5: [INF, INF, INF, 10, 6, 0, 9],
        6: [12, INF, 9, INF, 7, 9, INF]
    }

    problem = SalesmanSolver(list(range(num_of_houses)),
                             len(houses),
                             lambda x: decode_node(x),
                             lambda y: fit_salesman(y))
    cont = 0
    while cont <= POPULATION_SIZE:
        genetic_algorithm_t(problem, 2, min, 200, 100, RATIO, CROSS_PROB)
        cont += 1
    print("")


Chromosome: [5, 3, 1, 0, 6, 2, 4]
Solution: (['6', '4', '2', '1', '7', '3', '5'], 64)
Chromosome: [6, 4, 2, 0, 1, 3, 5]
Solution: (['7', '5', '3', '1', '2', '4', '6'], 63)
Chromosome: [4, 3, 5, 6, 0, 1, 2]
Solution: (['5', '4', '6', '7', '1', '2', '3'], 65)
Chromosome: [2, 1, 0, 6, 5, 3, 4]
Solution: (['3', '2', '1', '7', '6', '4', '5'], 65)
Chromosome: [1, 0, 2, 4, 6, 5, 3]
Solution: (['2', '1', '3', '5', '7', '6', '4'], 63)
Chromosome: [2, 4, 6, 5, 3, 1, 0]
Solution: (['3', '5', '7', '6', '4', '2', '1'], 63)
Chromosome: [5, 6, 4, 2, 0, 1, 3]
Solution: (['6', '7', '5', '3', '1', '2', '4'], 63)
Chromosome: [5, 6, 4, 2, 0, 1, 3]
Solution: (['6', '7', '5', '3', '1', '2', '4'], 63)
Chromosome: [1, 0, 6, 2, 4, 5, 3]
Solution: (['2', '1', '7', '3', '5', '6', '4'], 64)
Chromosome: [0, 6, 5, 3, 4, 2, 1]
Solution: (['1', '7', '6', '4', '5', '3', '2'], 65)
Chromosome: [3, 1, 0, 2, 4, 6, 5]
Solution: (['4', '2', '1', '3', '5', '7', '6'], 63)



# Q2

In [13]:
import numpy as np


class EquationSolver:
    """
    Using Genetic Algorithm
    """

    def __init__(self, equation, population_size=80, search_range=(-2000, 2000), epoch=100):
        """
        :param equation:
        :param population_size:
        :param search_range: at first we initially generate values and the values in this range.
        :param epoch:
        """

        # Assign initial values
        self.equation = equation
        self.population_size = population_size
        self.min_value, self.max_value = search_range
        self.epoch = epoch

        # Randomly initialize the first generation of the process
        self.population = []
        for i in range(self.population_size):
            v = np.random.randint(self.min_value, self.max_value, size=1)
            self.population.append(v)

    def give_birth(self):
        results = [self.equation(guy[0]) for guy in self.population]
        children = [self.population[np.argmin(np.absolute(results))]]

        while len(children) < self.population_size:
            ra, rb = [np.random.randint(0, self.population_size) for _ in range(2)]
            p1 = self.population[ra if results[ra] < results[rb] else rb]

            ra, rb = [np.random.randint(0, self.population_size) for _ in range(2)]
            p2 = self.population[ra if results[ra] < results[rb] else rb]

            s = self.get_sign(p1, p2)

            g1, g2 = self.produce_chromosome(p1, p2)
            children.append(np.asarray([s[0] * int(g1, 2), s[0] * int(g2, 2)]))

        self.population = children

    @staticmethod
    def get_sign(p1, p2):
        s = []
        for i in range(min(len(p2), len(p1))):
            if p1[i] < 0 and p2[i] < 0:
                s.append(-1)
            elif p1[i] >= 0 and p2[i] >= 0:
                s.append(1)
            else:
                s.append(np.random.choice([1, -1]))

        return s

    @staticmethod
    def produce_chromosome(p1, p2):
        chromosome_1 = [format(abs(i), '010b') for i in p1]
        chromosome_2 = [format(abs(i), '010b') for i in p2]

        c = []
        for i, j in zip(chromosome_1, chromosome_2):
            for k, l in zip(i, j):
                c.append(k if k == l else str(np.random.randint(min(k, l), max(k, l))))
        string = ''.join(c)

        # divide it to the same pieces to return the result
        half = len(string) // 2

        return string[0:half], string[half:len(string)]

    def run(self):
        for i in range(self.epoch):
            print("Generation's Index: {}".format(i))
            self.give_birth()

        return self.population[0][0]


In [14]:
# Test equations
SAMPLE_EQUATION_1 = lambda x: 9 * x ** 5 - 194.7 * x ** 4 + 1680.1 * x ** 3 - 7227.94 * x ** 2 + 15501.2 * x - 13257.2

if __name__ == '__main__':
    root = EquationSolver(
        SAMPLE_EQUATION_1,
        search_range=(-300, 300),
        epoch=300
    ).run()
    print("Root: {}\nValue in {} is {}".format(root, root, SAMPLE_EQUATION_1(root)))


Generation's Index: 0
Generation's Index: 1
Generation's Index: 2
Generation's Index: 3
Generation's Index: 4
Generation's Index: 5
Generation's Index: 6
Generation's Index: 7
Generation's Index: 8
Generation's Index: 9
Generation's Index: 10
Generation's Index: 11
Generation's Index: 12
Generation's Index: 13
Generation's Index: 14
Generation's Index: 15
Generation's Index: 16
Generation's Index: 17
Generation's Index: 18
Generation's Index: 19
Generation's Index: 20
Generation's Index: 21
Generation's Index: 22
Generation's Index: 23
Generation's Index: 24
Generation's Index: 25
Generation's Index: 26
Generation's Index: 27
Generation's Index: 28
Generation's Index: 29
Generation's Index: 30
Generation's Index: 31
Generation's Index: 32
Generation's Index: 33
Generation's Index: 34
Generation's Index: 35
Generation's Index: 36
Generation's Index: 37
Generation's Index: 38
Generation's Index: 39
Generation's Index: 40
Generation's Index: 41
Generation's Index: 42
Generation's Index: 4

# <font color='red'>Submission</font>

1. Sign up in [Gradescope](https://www.gradescope.com) with proper name and student ID and use the following code to join the class: <font color='red'>**D5372R**</font>
2. Fill in your full name (seperated by single spaces) and student ID in the beginning of this notebook.
3. After you're done with this notebook, you should do the following:
  - Clear all outputs of the notebook.
  ![clear all outputs](https://i.ibb.co/y6FrttB/Screen-Shot-2021-03-21-at-01-51-42.png)
  - Run all of the cells (if you skipped a question just leave the cell unchanged), and make sure all of your outputs are correct.
  ![run all](https://i.ibb.co/cgRcBZ0/Screen-Shot-2021-03-21-at-01-54-58.png)
  - Save your notebook.
  
  - If you're using Colab, download your notebook.
  ![download ipynb](https://i.ibb.co/2KxYM6K/Screen-Shot-2021-03-21-at-02-03-50.png)
  
  - Put the notebook file you just downloaded and `convert.py` in the same folder run the following command:
  ```bash
  python convert.py
  ```
  This will export your code for each question into a `.py` file.
   

  according to the question number.
  - There are 2 assignments in Gradescope: 

    You should upload your **codes** and your **notebook** in `HW4` section and your final report for all of the questions as a **single pdf** file in `HW4 - Report`. Autograder will automatically check for:
    - `CI992_HW4.ipynb`
    - `Q1 Q1.py`
    - `Q2 Q2.py`
    - Your name and ID in the beginning of `.ipynb` file.

    It is important that you <font color='red'>**don't**</font> change the names of these files before submission.

4. If you pass the autograder, you're good to go.